In [3]:
import glob
import os as os
import re as re
import pandas as pd
import matplotlib as mpl
import seaborn as sns

%reload_ext autoreload
%autoreload 2
%matplotlib inline
mpl.rc('text', usetex=True)
cmap = sns.color_palette("Paired", 10)

In [4]:
from motors.analysis import *
from motors.aesthetics import *
prepare_plot()

# Scan concentrations

In [4]:
calculation = False

The general strategy is:

1. Specify the directory
1. Get a list of files and exclude a few that aren't rotatable
1. For each concentration, get the directional flux $J$, the intersurface flux, the reciprocating flux $J_R$, the maximum power $P_\text{max}$, and the load at the maximum power $\tau_\text{stall}/2$
1. Add these to a growing data frame
1. Save the pickle.

## PKA

In [5]:
pka_dir = './md-data/pka-md-data/'
pka_unbound_files = sorted(glob.glob(pka_dir + 'apo/' + '*'))
names = []
for file in range(len(pka_unbound_files)):
    name = os.path.splitext(os.path.basename(pka_unbound_files[file]))[0]
    name = re.search('^[^_]*', name).group(0)        
    if re.search('omega*', name):
        continue
    if re.search('chi3ASN*', name):
        continue
    if re.search('chi5LYS*', name):
        continue
    # This one is missing in the PKA data set.
    if re.search('chi1CYM185', name):
        continue
    names.append(name)

In [12]:
df = pd.DataFrame()
if calculation:
    for concentration in np.arange(-6, 0, 0.1):
        for name in names:
            directional_flux, intersurface_flux, driven_flux = summarize_fluxes(name, concentration=10**concentration,
                                                                               data_source='pka_md_data') 
            if directional_flux < 0:
                negative=True
            else:
                negative=False
            max_power, max_load = summarize_power_and_load(name, concentration=10**concentration, negative=negative, 
                                                           debug=False, data_source='pka_md_data')
            df = df.append(pd.DataFrame({'Concentration': concentration,
                                    'Directional flux': directional_flux,
                                    'Intersurface flux': intersurface_flux,
                                    'Driven flux': driven_flux,
                                    'File': name,
                                    'Max power': max_power,
                                    'Max load' : max_load,
                                    'ResID': re.match('.*?([0-9]+)$', name).group(1),
                                    }, index=[0]), ignore_index=True)

    df.to_pickle('pka-concentration-scan.pickle')

In [1]:
calculation = True

In [5]:
hiv_dir = './md-data/hiv-protease/'
hiv_unbound_files = sorted(glob.glob(hiv_dir + '1hhp_apo/' + '*'))
names = []
for file in range(len(hiv_unbound_files)):
    name = os.path.splitext(os.path.basename(hiv_unbound_files[file]))[0]
    name = re.search('^[^_]*', name).group(0)        
    if re.search('omega*', name):
        continue
    names.append(name)

In [6]:
df = pd.DataFrame()
if calculation:
    for concentration in np.arange(-6, 0, 0.1):
        for name in names:
            directional_flux, intersurface_flux, driven_flux = summarize_fluxes(name, concentration=10**concentration,
                                                                                catalytic_rate=0.3,
                                                                                data_source='hiv_md_data') 
            if directional_flux < 0:
                negative=True
            else:
                negative=False
            df = df.append(pd.DataFrame({'Concentration': concentration,
                                    'Directional flux': directional_flux,
                                    'Intersurface flux': intersurface_flux,
                                    'Driven flux': driven_flux,
                                    'File': name,
                                    'ResID': re.match('.*?([0-9]+)$', name).group(1),
                                    }, index=[0]), ignore_index=True)

    df.to_pickle('precomputed-pickles/hiv-concentration-scan-catalytic-rate-0point3.pickle')

In [7]:
df = pd.DataFrame()
if calculation:
    for concentration in np.arange(-6, 0, 0.1):
        for name in names:
            directional_flux, intersurface_flux, driven_flux = summarize_fluxes(name, concentration=10**concentration,
                                                                                catalytic_rate=10.0,
                                                                                data_source='hiv_md_data') 
            if directional_flux < 0:
                negative=True
            else:
                negative=False
            df = df.append(pd.DataFrame({'Concentration': concentration,
                                    'Directional flux': directional_flux,
                                    'Intersurface flux': intersurface_flux,
                                    'Driven flux': driven_flux,
                                    'File': name,
                                    'ResID': re.match('.*?([0-9]+)$', name).group(1),
                                    }, index=[0]), ignore_index=True)

    df.to_pickle('precomputed-pickles/hiv-concentration-scan-catalytic-rate-10v2.pickle')

In [9]:
df.sort_values('Directional flux', ascending=True).head(2)

,Concentration,Directional flux,Driven flux,File,Intersurface flux,ResID
45277,-0.6,-5.620324,8.480225,chi2ASP124,1.590913,124
46112,-0.5,-5.618817,8.478857,chi2ASP124,1.590684,124


In [10]:
df.sort_values('Directional flux', ascending=False).head(2)

,Concentration,Directional flux,Driven flux,File,Intersurface flux,ResID
44196,-0.8,0.583228,1.952357,psiLYS55,0.453791,55
35846,-1.8,0.552838,1.850525,psiLYS55,0.463735,55


In [12]:
df = pd.read_pickle('precomputed-pickles/hiv-concentration-scan-catalytic-rate-0point3.pickle')

In [13]:
df.sort_values('Directional flux', ascending=False).head(2)

,Concentration,Directional flux,Driven flux,File,Intersurface flux,ResID
42526,-1.0,0.703236,2.625626,psiLYS55,0.612504,55
46701,-0.5,0.693207,2.830953,psiLYS55,0.599938,55


In [14]:
df.sort_values('Directional flux', ascending=True).head(2)

,Concentration,Directional flux,Driven flux,File,Intersurface flux,ResID
40021,-1.3,-0.934942,2.196478,psiLYS55,0.069352,55
45866,-0.6,-0.516835,2.226444,psiLYS55,0.590442,55


In [15]:
df[df['File'] == 'chi2ASP124']

,Concentration,Directional flux,Driven flux,File,Intersurface flux,ResID
187,-6.0,0.000037,0.000717,chi2ASP124,0.000087,124
1022,-5.9,-0.000152,0.001293,chi2ASP124,0.000109,124
1857,-5.8,-0.000418,0.000716,chi2ASP124,0.000136,124
2692,-5.7,-0.000559,0.000798,chi2ASP124,0.000176,124
3527,-5.6,-0.000228,0.000951,chi2ASP124,0.000221,124
4362,-5.5,-0.000772,0.001666,chi2ASP124,0.000264,124
5197,-5.4,-0.000644,0.001451,chi2ASP124,0.000355,124
6032,-5.3,-0.001105,0.002258,chi2ASP124,0.000433,124
6867,-5.2,-0.002132,0.004326,chi2ASP124,0.000531,124
7702,-5.1,-0.002212,0.003295,chi2ASP124,0.000664,124
